In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from expected_tackling.data.mott_features import compute_mott_features_data

In [3]:
features_data = pd.concat([pd.read_csv(f"../data/features_week_{i}.csv") for i in range(1,10)]).reset_index(drop=True)
tackling_probability = pd.read_csv("../data/tackling_probability.csv")
tackles = pd.read_csv("../data/tackles.csv")

In [104]:
mott_features_data = compute_mott_features_data(features_data, tackling_probability, tackles)

In [105]:
mott_features_data

ott  nb_frames_from_end  \
gameId     playId nflId                                   
2022090800 56     38577.0  0.158333                 3.0   
                  41239.0  0.000607                 8.0   
                  42816.0  0.001262                12.0   
                  43294.0  1.046483                 0.0   
                  43298.0  0.000308                16.0   
...                             ...                 ...   
2022110700 3787   52627.0  0.012055                15.0   
                  53460.0  0.115559                17.0   
                  53533.0  0.001109                 0.0   
                  54541.0  1.136388                 7.0   
                  54606.0  0.005385                 3.0   

                           mean_distance_to_ball_carrier_from_peak  \
gameId     playId nflId                                              
2022090800 56     38577.0                                 1.971121   
                  41239.0                                13.907379   
                  42816.0                                27.279112   
                  43294.0                                 0.670298   
                  43298.0                                12.350122   
...                                                            ...   
2022110700 3787   52627.0                                13.470441   
                  53460.0                                 4.405723   
                  53533.0                                19.715603   
                  54541.0                                 1.121133   
                  54606.0                                12.675541   

                           ball_carrier_distance_won_to_last_frame  \
gameId     playId nflId                                              
2022090800 56     38577.0                                     0.00   
                  41239.0                                     0.00   
                  42816.0                                     0.52   
                  43294.0                                     0.00   
                  43298.0                                     1.15   
...                                                            ...   
2022110700 3787   52627.0                                     5.57   
                  53460.0                                     6.54   
                  53533.0                                     0.00   
                  54541.0                                     1.95   
                  54606.0                                     0.59   

                           tackle_or_assist  pff_missedTackle  
gameId     playId nflId                                        
2022090800 56     38577.0               0.0               0.0  
                  41239.0               0.0               0.0  
                  42816.0               0.0               0.0  
                  43294.0               1.0               0.0  
                  43298.0               0.0               0.0  
...                                     ...               ...  
2022110700 3787   52627.0               0.0               0.0  
                  53460.0               0.0               0.0  
                  53533.0               0.0               0.0  
                  54541.0               0.0               0.0  
                  54606.0               0.0               0.0  

[127941 rows x 6 columns]

In [129]:
sample_mott_features_data = pd.concat([
    mott_features_data[(mott_features_data["pff_missedTackle"]==1)&(mott_features_data["ott"]>0.1)],
    mott_features_data[mott_features_data["pff_missedTackle"]==0].sample(
        len(mott_features_data[mott_features_data["pff_missedTackle"]==1])*10
    )
])

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from catboost import CatBoostClassifier

In [131]:
X_train, X_test, y_train, y_test = train_test_split(
    sample_mott_features_data.drop(columns=["pff_missedTackle"]), 
    sample_mott_features_data["pff_missedTackle"], 
    test_size=0.33, 
    random_state=42
)

In [132]:
model = CatBoostClassifier(max_depth=2, scale_pos_weight=5, verbose=0)
model.fit(X_train, y_train)

In [133]:
y_pred = model.predict(X_test)

In [134]:
y_test.sum()/len(y_test)

0.08416188208934323

In [135]:
y_pred.sum()/len(y_pred)

0.10726788965130934

In [136]:
balanced_accuracy_score(y_train, model.predict(X_train))

0.9482267027536487

In [137]:
balanced_accuracy_score(y_test, y_pred)

0.948324148844688

In [138]:
confusion_matrix(y_train, model.predict(X_train))

array([[12796,   384],
       [   98,  1219]], dtype=int64)

In [139]:
confusion_matrix(y_test, y_pred)

array([[6332,  208],
       [  43,  558]], dtype=int64)

In [140]:
import pickle
pickle.dump(model, open("../models/model_mott.pkl", 'wb'))

In [141]:
mott_features_data.to_csv("../data/features_mott.csv")

In [142]:
result_data = mott_features_data.copy()
result_data["prediction"] = model.predict(result_data.drop(columns=["pff_missedTackle"]))

In [143]:
result_data

ott  nb_frames_from_end  \
gameId     playId nflId                                   
2022090800 56     38577.0  0.158333                 3.0   
                  41239.0  0.000607                 8.0   
                  42816.0  0.001262                12.0   
                  43294.0  1.046483                 0.0   
                  43298.0  0.000308                16.0   
...                             ...                 ...   
2022110700 3787   52627.0  0.012055                15.0   
                  53460.0  0.115559                17.0   
                  53533.0  0.001109                 0.0   
                  54541.0  1.136388                 7.0   
                  54606.0  0.005385                 3.0   

                           mean_distance_to_ball_carrier_from_peak  \
gameId     playId nflId                                              
2022090800 56     38577.0                                 1.971121   
                  41239.0                                13.907379   
                  42816.0                                27.279112   
                  43294.0                                 0.670298   
                  43298.0                                12.350122   
...                                                            ...   
2022110700 3787   52627.0                                13.470441   
                  53460.0                                 4.405723   
                  53533.0                                19.715603   
                  54541.0                                 1.121133   
                  54606.0                                12.675541   

                           ball_carrier_distance_won_to_last_frame  \
gameId     playId nflId                                              
2022090800 56     38577.0                                     0.00   
                  41239.0                                     0.00   
                  42816.0                                     0.52   
                  43294.0                                     0.00   
                  43298.0                                     1.15   
...                                                            ...   
2022110700 3787   52627.0                                     5.57   
                  53460.0                                     6.54   
                  53533.0                                     0.00   
                  54541.0                                     1.95   
                  54606.0                                     0.59   

                           tackle_or_assist  pff_missedTackle  prediction  
gameId     playId nflId                                                    
2022090800 56     38577.0               0.0               0.0         0.0  
                  41239.0               0.0               0.0         0.0  
                  42816.0               0.0               0.0         0.0  
                  43294.0               1.0               0.0         0.0  
                  43298.0               0.0               0.0         0.0  
...                                     ...               ...         ...  
2022110700 3787   52627.0               0.0               0.0         0.0  
                  53460.0               0.0               0.0         0.0  
                  53533.0               0.0               0.0         0.0  
                  54541.0               0.0               0.0         1.0  
                  54606.0               0.0               0.0         0.0  

[127941 rows x 7 columns]

In [144]:
result_data.loc[(2022100911,2915)]

,ott,nb_frames_from_end,mean_distance_to_ball_carrier_from_peak,ball_carrier_distance_won_to_last_frame,tackle_or_assist,pff_missedTackle,prediction
nflId,,,,,,,
42401.0,0.000421,24.0,19.369486,7.07,0.0,0.0,0.0
43986.0,0.795232,20.0,4.820028,6.44,0.0,1.0,1.0
44848.0,0.018716,3.0,8.442979,1.66,0.0,0.0,0.0
47816.0,1.076127,12.0,3.278709,5.02,1.0,0.0,1.0
47848.0,0.001377,29.0,16.561179,7.89,0.0,0.0,0.0
49410.0,0.012372,6.0,9.991531,3.07,0.0,0.0,0.0
52416.0,0.605927,7.0,2.162725,3.48,0.0,0.0,1.0
52522.0,0.003440,26.0,10.140932,7.43,0.0,0.0,0.0
53445.0,0.032192,0.0,5.606291,0.00,0.0,0.0,0.0
